# Visualiser des données géographiques avec la librairie Folium

Pouvoir représenter une carte est souvent une composante essentielle d'une analyse de données. 

La librairie **Folium** permet de créer facilement des cartes interactives. Vous pouvez en lire plus sur la documentation officielle - https://www.pypi.python.org/pypi/folium

Dans cette partie, vous verrez : 

- Comment intégrer une carte interactive au sein de l'environnement iPython
- Comment tracer des points à partir d"un DataFrame 
- Comment tracer des zones, en utilisant des repères TopoJSON

## Créer des cartes et des repères

Installez la librairie folium depuis votre terminal grâce à pip : 

Nous importons tout d'abord la librairie folium ainsi que le module HTML de iPython :

In [1]:
import folium
import json
from geojson_utils import point_in_polygon

Nous définissons les paramètres d'affichage de notre carte en précisant son centre:

In [95]:
m = folium.Map(location=[43.6,1.43333])
m

Nous pouvons ajouter un repère grâce aux méthodes `Marker`, `CircleMarket`: 

In [96]:
folium.CircleMarker(
    location=[48.89157,2.24099],
    radius=50,
    color='#3186cc',
).add_to(m)

m

In [97]:
folium.Marker(
    location=[48.89157,2.24099],
).add_to(m)

m

Nous pouvons enfin ajouter du contenu au sein de la pop up des marqueurs que nous avons créé, en passant un chaine de caractère au sein de la méthode **popup**

In [ ]:
folium.Marker(
    location=[48.89157,2.24099],
    popup='Hello!',
).add_to(m)

m

## Créer des repères à partir d'un DataFrame

Nous pouvons donc appliquer la même méthode à partir de données issues d'un DataFrame. 

Nous pouvons par exemple afficher une carte de France des emplacements des restaurants McDonalds. 

Nous importons pandas : 

In [2]:
import pandas as pd

Nous chargeons le fichier McDonalds_Fr. Celui-ci contient les latitudes, longitudes et nom de tous les McDonalds de France : 

In [3]:
durations = pd.read_csv("data/durations_avoid_toll.csv",encoding='latin-1')
durations.head(3)

,Unnamed: 0,ville_code_commune,duration_toulouse,distance_toulouse,base_duration_toulouse
0,0,9045,5796,104006,5605
1,1,9098,5818,91446,5676
2,2,9302,6938,121621,6814


Comme précédemment, nous déclarons une carte Folium : 

In [3]:
map_durations= folium.Map(location=[43.6,1.43333], 
                     zoom_start=5, 
                     tiles='Stamen Toner')

Nous réalisons une boucle à partir des valeurs de notre DataFrame :

In [ ]:
for x in mcdo.values:
    location = []
    location.append(x[1])
    location.append(x[0])
    
    folium.CircleMarker(
        location=location,
        radius=4,
        color='red',
        popup=x[2]
    ).add_to(map_mcdo)
    

Nous appelons enfin la méthode **inline_map** pour afficher notre carte : 

In [ ]:
map_mcdo

## Créer des zones géographiques à partir de pandas et des fichiers GeoJSON

Folium vous permet également de créer vos propres polygônes géographiques en se basant sur les standard TopoJSON / GeoJSON. Comme leur nom l'indique, ces fichiers sont des fichiers JSON décrivant les frontières d'une zone géographique à tracer. 

Il existe de nombreuses ressources en ligne permettant de récupérer des fichiers TopoJSON existant ou de construire les siens. 

Nous prendrons dans cet exemple des fichiers issus de ce repo GitHub - https://github.com/gregoiredavid/france-geojson où sont disponibles les communes, les départements et les régions. 

Nous allons dans cet exemple, réaliser une carte du taux de chômage en France, par département. 

La méthode **GeoJSON** permet d'afficher une zone géographique sur une carte. Il suffit dans ce cas de passer en argument le fichier TopoJSON source : 

In [11]:
map_durations = folium.Map(location=[43.6,1.43333],
                             zoom_start=9)

L'une des fonctionnalités assez sympa de Folium est la possibilité de s'interfacer avec un DataFrame Pandas. Voyez plutôt : 

Nous créons un DataFrame **dep_data** à partir des données de chômage dont nous disposons :

In [4]:
durations = pd.read_csv("data/durations_avoid_toll.csv", sep=',', error_bad_lines=False, index_col=False, dtype='str')
durations['base_duration_toulouse'] = durations['base_duration_toulouse'].astype('int')

In [5]:
durations.head()

,Unnamed: 0,ville_code_commune,duration_toulouse,distance_toulouse,base_duration_toulouse
0,0,09045,5796,104006,5605
1,1,09098,5818,91446,5676
2,2,09302,6938,121621,6814
3,3,09327,5291,87831,5174
4,4,09310,3786,62644,3595


In [6]:
durations['base_duration_toulouse'] = abs(durations['base_duration_toulouse']-4200)/60
durations['base_duration_toulouse'] = durations['base_duration_toulouse'].astype('int')
durations.head()

,Unnamed: 0,ville_code_commune,duration_toulouse,distance_toulouse,base_duration_toulouse
0,0,09045,5796,104006,23
1,1,09098,5818,91446,24
2,2,09302,6938,121621,43
3,3,09327,5291,87831,16
4,4,09310,3786,62644,10


Nous réalisons ensuite le binding entre les données de notre DataFrame et nos départements. 

Plusieurs éléments sont à noter: 
    
- Nous sélectionnons au sein de **column**, deux colonnes. La première doit présenter une clé commune avec notre fichier TopoJSON
- Le paramètre **key_on** précise la clé à utiliser, au sein du fichier TopoJSON pour lier nos données.  

In [7]:
durations_1h10 = durations.loc[durations['base_duration_toulouse'].between(0, 30)]
durations_1h10.count()

Unnamed: 0                1421
ville_code_commune        1421
duration_toulouse         1421
distance_toulouse         1421
base_duration_toulouse    1421
dtype: int64

In [9]:
multi_polygone = {
    "type": "Polygon",
    "coordinates": []
} 
with open('data/communes-occitanie.geojson','r') as fp:
    communes_cc = json.load(fp)
    
print(communes_cc['features'][0])

#communes_cc['features'] = [f for f['properties']['code'] in durations_1h10['ville_code_commune'].values] 

new_feats=[]
print(durations_1h10['ville_code_commune'].values)
for f in communes_cc['features']:
    if f['properties']['code'] in durations_1h10['ville_code_commune'].values:
        new_feats.append(f)
        multi_polygone['coordinates'].append(f['geometry']['coordinates'])
        
communes_cc['features'] = new_feats
print(len(communes_cc['features']))

{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[1.05055, 43.54104], [1.05299, 43.54097], [1.05942, 43.53893], [1.06136, 43.54056], [1.06551, 43.54043], [1.06651, 43.53991], [1.07015, 43.54287], [1.07151, 43.54622], [1.07823, 43.54733], [1.08239, 43.54647], [1.08771, 43.5469], [1.08768, 43.5455], [1.09065, 43.54521], [1.09127, 43.54199], [1.09266, 43.54033], [1.0973, 43.53755], [1.09474, 43.53585], [1.09708, 43.53322], [1.09131, 43.53158], [1.08902, 43.5312], [1.08746, 43.52916], [1.08761, 43.52457], [1.0836, 43.52487], [1.07953, 43.52694], [1.07641, 43.52692], [1.07522, 43.52867], [1.07306, 43.52815], [1.07056, 43.52965], [1.06807, 43.53308], [1.06694, 43.53392], [1.06702, 43.5359], [1.05928, 43.53681], [1.05801, 43.53783], [1.0534, 43.53962], [1.05055, 43.54104]]]}, 'properties': {'code': '31166', 'nom': 'Empeaux'}}
['09045' '09098' '09327' ... '82126' '82139' '82093']
1417


In [12]:
with open('data/geo_1h10_avoid_toll.json', 'w') as f:
    json.dump(communes_cc, f)

#    'data/geo_1h10.json'
folium.GeoJson(communes_cc).add_to(map_durations)

In [13]:
map_durations